# S2RA_ETBF_doc_test_export

**Date:** <br>
23 February 2022 <br>
**Background:** <br>
Issue -  <br>
**Author(s):**<br>
Thomas Moore<br>

### OOD documentation
https://opus.nci.org.au/display/DAE/Setting+up+a+Dask+Cluster+on+OOD

In [1]:
Author1 = {"name": "Thomas Moore", "affiliation": "CSIRO", "email": "thomas.moore@csiro.au", "orcid": "0000-0003-3930-1946"}

In [2]:
import xarray as xr
import numpy as np
import xrft
import xesmf as xe
import scipy
import matplotlib.pyplot as plt
import datetime
import pandas as pd
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
import os
import re
import cartopy.crs as ccrs
import proplot as pplt
from rechunker import rechunk
%config Completer.use_jedi = False

## import helper

In [3]:
import importlib.util
spec = importlib.util.spec_from_file_location("helper", "/g/data/v14/tm4888/code/helper-py/helper_tools.py")
helper = importlib.util.module_from_spec(spec)
spec.loader.exec_module(helper)

## OOD cluster

In [4]:
from dask.distributed import Client,Scheduler
from dask_jobqueue import SLURMCluster
cluster = SLURMCluster(cores=1,processes=1,memory="47GB",walltime='02:00:00')
client = Client(cluster)
cluster.scale(cores=24)

/g/data/v14/tm4888/miniconda3/envs/pangeo_hpc/lib/python3.9/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


In [6]:
client

<Client: 'tcp://10.0.128.132:43267' processes=16 threads=16, memory=700.32 GiB>

# load final ETBF S2 025grid collections from zarr

In [7]:
%%time
ds = xr.open_zarr('/g/data/v14/tm4888/data/ACCESS-S2/accessS2.RA.ocean.masked.AUSWCPregion.ETBFvars.zarr',consolidated=True)

CPU times: user 161 ms, sys: 44.4 ms, total: 206 ms
Wall time: 400 ms


In [8]:
ds = ds.drop_vars(['areat','areau','areav','maskT','maskU','maskV'])
ds

<xarray.Dataset>
Dimensions:        (time: 492, lat: 361, lon: 721)
Coordinates:
  * lat            (lat) float64 -70.0 -69.75 -69.5 -69.25 ... 19.5 19.75 20.0
  * lon            (lon) float64 90.0 90.25 90.5 90.75 ... 269.5 269.8 270.0
  * time           (time) datetime64[ns] 1981-01-15 1981-02-15 ... 2021-12-15
Data variables: (12/18)
    d20            (time, lat, lon) float32 dask.array<chunksize=(12, 361, 721), meta=np.ndarray>
    eke2000        (time, lat, lon) float32 dask.array<chunksize=(1, 361, 721), meta=np.ndarray>
    eke300         (time, lat, lon) float32 dask.array<chunksize=(1, 361, 721), meta=np.ndarray>
    hc300          (time, lat, lon) float32 dask.array<chunksize=(12, 361, 721), meta=np.ndarray>
    mld1           (time, lat, lon) float32 dask.array<chunksize=(12, 361, 721), meta=np.ndarray>
    mld2           (time, lat, lon) float32 dask.array<chunksize=(12, 361, 721), meta=np.ndarray>
    ...             ...
    temp50         (time, lat, lon) float32 dask.array<chunksize=(12, 361, 721), meta=np.ndarray>
    temp500        (time, lat, lon) float32 dask.array<chunksize=(12, 361, 721), meta=np.ndarray>
    u100           (time, lat, lon) float32 dask.array<chunksize=(1, 361, 721), meta=np.ndarray>
    u100_300       (time, lat, lon) float32 dask.array<chunksize=(1, 361, 721), meta=np.ndarray>
    v100           (time, lat, lon) float32 dask.array<chunksize=(1, 361, 721), meta=np.ndarray>
    v100_300       (time, lat, lon) float32 dask.array<chunksize=(1, 361, 721), meta=np.ndarray>
Attributes:
    regrid_method:  bilinear

# Tests $\Downarrow$

In [ ]:
# Test for zeroes
for varname, da in ds.data_vars.items():
    if (ds[varname] == 0).any():
        raise ValueError(varname + " variable has a zero value <<<")
    else:
        print('No zero values found in '+varname)

In [ ]:
# Test for negative values
for varname, da in ds.data_vars.items():
    if (ds[varname] < 0).any():
        print(varname + " variable has negative values <<<")
    else:
        print('No negative values found in '+varname)

# Tests $\Uparrow$

# 

In [9]:
ds

<xarray.Dataset>
Dimensions:        (time: 492, lat: 361, lon: 721)
Coordinates:
  * lat            (lat) float64 -70.0 -69.75 -69.5 -69.25 ... 19.5 19.75 20.0
  * lon            (lon) float64 90.0 90.25 90.5 90.75 ... 269.5 269.8 270.0
  * time           (time) datetime64[ns] 1981-01-15 1981-02-15 ... 2021-12-15
Data variables: (12/18)
    d20            (time, lat, lon) float32 dask.array<chunksize=(12, 361, 721), meta=np.ndarray>
    eke2000        (time, lat, lon) float32 dask.array<chunksize=(1, 361, 721), meta=np.ndarray>
    eke300         (time, lat, lon) float32 dask.array<chunksize=(1, 361, 721), meta=np.ndarray>
    hc300          (time, lat, lon) float32 dask.array<chunksize=(12, 361, 721), meta=np.ndarray>
    mld1           (time, lat, lon) float32 dask.array<chunksize=(12, 361, 721), meta=np.ndarray>
    mld2           (time, lat, lon) float32 dask.array<chunksize=(12, 361, 721), meta=np.ndarray>
    ...             ...
    temp50         (time, lat, lon) float32 dask.array<chunksize=(12, 361, 721), meta=np.ndarray>
    temp500        (time, lat, lon) float32 dask.array<chunksize=(12, 361, 721), meta=np.ndarray>
    u100           (time, lat, lon) float32 dask.array<chunksize=(1, 361, 721), meta=np.ndarray>
    u100_300       (time, lat, lon) float32 dask.array<chunksize=(1, 361, 721), meta=np.ndarray>
    v100           (time, lat, lon) float32 dask.array<chunksize=(1, 361, 721), meta=np.ndarray>
    v100_300       (time, lat, lon) float32 dask.array<chunksize=(1, 361, 721), meta=np.ndarray>
Attributes:
    regrid_method:  bilinear

# write metadata

In [10]:
#global
ds.attrs = {'regrid_method':'xESMF - bilnear + nearest_s2d extrapolation',
           'product' : 'BOM ACCESS-S2 Reanalysis - Seasonal Prediction ACCESS-S2 Hindcast (1981-2018) and Supporting Data Assimilation and Initial Conditions',
           'product URL': 'https://geonetwork.nci.org.au/geonetwork/srv/eng/catalog.search#/metadata/f3311_4920_0252_8073',
           'contact':'thomas.moore@csiro.au'}
#coordinates
ds['lat'].attrs = {'long_name':'latitude','units':'degrees'}
ds['lon'].attrs = {'long_name':'longitude','units':'degrees'}
#ds['time'].attrs = {'long_name':'timestamp','units':'month'}
#variables
ds['d20'].attrs = {'long_name':'Depth of 20C isotherm','units':'m'}
ds['eke2000'].attrs = {'long_name':'eke depth weighted sum 0-2000m'}
ds['eke300'].attrs = {'long_name':'eke depth weighted sum 0-300m'}
ds['hc300'].attrs = {'long_name':'Heat content 300 m','units':'J/m2'}
ds['mld1'].attrs = {'long_name':'Kara Mixed Layer Depth','units':'m'}
ds['mld2'].attrs = {'long_name':'Mixed Layer Depth 0.01','units':'m'}
ds['ssh_corrected'].attrs = {'long_name':'Sea Surface Height','units':'m'}
ds['sss'].attrs = {'long_name':'Sea Surface Salinity','units':'PSU'}
ds['sst'].attrs = {'long_name':'Sea Surface Temperature','units':'degC'}
ds['td'].attrs = {'long_name':'Thermocline Depth','units':'m'}
ds['temp50'].attrs = {'long_name':'Seawater Potential Temperature at 50m','units':'degC'}
ds['temp100'].attrs = {'long_name':'Seawater Potential Temperature at 100m','units':'degC'}
ds['temp200'].attrs = {'long_name':'Seawater Potential Temperature at 200m','units':'degC'}
ds['temp500'].attrs = {'long_name':'Seawater Potential Temperature at 500m','units':'degC'}
ds['u100'].attrs = {'long_name':'x velocity at 100m','units':'m/s'}
ds['v100'].attrs = {'long_name':'y velocity at 100m','units':'m/s'}
ds['u100_300'].attrs = {'long_name':'x velocity depth weighted mean between 100-300m','units':'m/s'}
ds['v100_300'].attrs = {'long_name':'y velocity depth weighted mean between 100-300m','units':'m/s'}
ds

<xarray.Dataset>
Dimensions:        (time: 492, lat: 361, lon: 721)
Coordinates:
  * lat            (lat) float64 -70.0 -69.75 -69.5 -69.25 ... 19.5 19.75 20.0
  * lon            (lon) float64 90.0 90.25 90.5 90.75 ... 269.5 269.8 270.0
  * time           (time) datetime64[ns] 1981-01-15 1981-02-15 ... 2021-12-15
Data variables: (12/18)
    d20            (time, lat, lon) float32 dask.array<chunksize=(12, 361, 721), meta=np.ndarray>
    eke2000        (time, lat, lon) float32 dask.array<chunksize=(1, 361, 721), meta=np.ndarray>
    eke300         (time, lat, lon) float32 dask.array<chunksize=(1, 361, 721), meta=np.ndarray>
    hc300          (time, lat, lon) float32 dask.array<chunksize=(12, 361, 721), meta=np.ndarray>
    mld1           (time, lat, lon) float32 dask.array<chunksize=(12, 361, 721), meta=np.ndarray>
    mld2           (time, lat, lon) float32 dask.array<chunksize=(12, 361, 721), meta=np.ndarray>
    ...             ...
    temp50         (time, lat, lon) float32 dask.array<chunksize=(12, 361, 721), meta=np.ndarray>
    temp500        (time, lat, lon) float32 dask.array<chunksize=(12, 361, 721), meta=np.ndarray>
    u100           (time, lat, lon) float32 dask.array<chunksize=(1, 361, 721), meta=np.ndarray>
    u100_300       (time, lat, lon) float32 dask.array<chunksize=(1, 361, 721), meta=np.ndarray>
    v100           (time, lat, lon) float32 dask.array<chunksize=(1, 361, 721), meta=np.ndarray>
    v100_300       (time, lat, lon) float32 dask.array<chunksize=(1, 361, 721), meta=np.ndarray>
Attributes:
    regrid_method:  xESMF - bilnear + nearest_s2d extrapolation
    product:        BOM ACCESS-S2 Reanalysis - Seasonal Prediction ACCESS-S2 ...
    product URL:    https://geonetwork.nci.org.au/geonetwork/srv/eng/catalog....
    contact:        thomas.moore@csiro.au

# Write out single NC file

In [11]:
%%time
ds.to_netcdf('/g/data/v14/tm4888/data/ACCESS-S2/ETBF_export/AUS_region/accessS2.RA.ETBFvars.AUSregion.grid025deg.nc')

CPU times: user 2min 5s, sys: 11.6 s, total: 2min 17s
Wall time: 7min 21s


# $ The\ End$

# Break glass in case of emergency
# $\Downarrow$

In [ ]:
client.restart()

In [ ]:
client.shutdown()

In [ ]:
client.restart()